# Wrangle Notebook

## Step 1 | Acquiring the Data
> `MVI` is Maternal Vulnerability Index, a scale from 0 - 100 that represents the vulnerability score of mothers, with 0 being the least vulnerable and 100 being the most vulnerable.

Data Sources 
- MVI - County (csv) | MVI data by county
- MVI - State (csv) | MVI data state
- ACS_2017 County Data (csv) | Demographics by county
- US Counties (csv) | Geodata and population by county

Info files
- MVI Data Architecture and Sources
- MVI Data Dictionary

In [1]:
import pandas as pd

pd.set_option('display.max_columns', None) #to prevent df columns from being truncated (scroll :)

#### Reading in the `mvi_county` csv file to a df

In [2]:
# MVI - County (csv) | MVI
df_mvi_county = pd.read_csv('mvi_county.csv')
df_mvi_county.head(2)

,GEOID,State,state_GEOID,county_name,mvi,Th1,Th2,Th3,Th4,Th5,Th6
0,2282,Alaska,2,Yakutat,48.58,17.38,52.98,36.55,94.52,16.43,78.32
1,2060,Alaska,2,Bristol Bay,41.17,27.76,64.85,30.31,71.47,13.66,58.90


In [3]:
print(f'There are {df_mvi_county.shape[0]} rows and {df_mvi_county.shape[1]} columns in the mvi_county data')
print(f'There are {df_mvi_county.isna().sum().sum()} null values.')

There are 3142 rows and 11 columns in the mvi_county data
There are 0 null values.


#### Reading in the `county_data` csv file to a df

In [4]:
# ACS_2017 County Data (csv) | Demographics
df_county_data = pd.read_csv('acs2017_county_data.csv')
df_county_data.head(2)


,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001,Alabama,Autauga County,55036,26899,28137,2.7,75.4,18.9,0.3,0.9,0.0,41016,55317,2838,27824,2024,13.7,20.1,35.3,18.0,23.2,8.1,15.4,86.0,9.6,0.1,0.6,1.3,2.5,25.8,24112,74.1,20.2,5.6,0.1,5.2
1,1003,Alabama,Baldwin County,203360,99527,103833,4.4,83.1,9.5,0.8,0.7,0.0,155376,52562,1348,29364,735,11.8,16.1,35.7,18.2,25.6,9.7,10.8,84.7,7.6,0.1,0.8,1.1,5.6,27.0,89527,80.7,12.9,6.3,0.1,5.5


In [5]:
print(f'There are {df_county_data.shape[0]} rows and {df_county_data.shape[1]} columns in the mvi_county data')
print(f'There are {df_county_data.isna().sum().sum()} null values.')

There are 3220 rows and 37 columns in the mvi_county data
There are 1 null values.


In [6]:
df_county_data.isna().sum()

CountyId            0
State               0
County              0
TotalPop            0
Men                 0
Women               0
Hispanic            0
White               0
Black               0
Native              0
Asian               0
Pacific             0
VotingAgeCitizen    0
Income              0
IncomeErr           0
IncomePerCap        0
IncomePerCapErr     0
Poverty             0
ChildPoverty        1
Professional        0
Service             0
Office              0
Construction        0
Production          0
Drive               0
Carpool             0
Transit             0
Walk                0
OtherTransp         0
WorkAtHome          0
MeanCommute         0
Employed            0
PrivateWork         0
PublicWork          0
SelfEmployed        0
FamilyWork          0
Unemployment        0
dtype: int64

#### There is one null value in the `ChildPoverty` field. Let's take a look....

In [7]:
df_county_data[df_county_data.ChildPoverty.isna() == True]

,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
548,15005,Hawaii,Kalawao County,86,41,45,4.7,20.9,0.0,0.0,29.1,33.7,84,61750,11280,46024,7959,12.7,NaN,22.2,38.1,20.6,0.0,19.0,32.8,9.4,0.0,40.6,10.9,6.3,6.6,63,34.9,61.9,3.2,0.0,0.0


#### ...since this is just one observation, will simply drop this column for the MVP

In [8]:
df_county_data.dropna(inplace = True)

#### Looking at the data I can see that the `mvi` dataset provides lots of great information about the actual vulnerability scores and the `county_data` has lots of great demographic data that can be used to further explore what could be driving the mvi scores, along with the various sub-scores that are direct drivers of the mvi.
Both tables have a column with the FIPS Codes for each county and I can use this to join them, performing an `inner_join` to eliminate having to wrangle any nulls this first pass for the mvp.
>I am going to set the `mvi` df index as `GEOID` and the `county_data` index as `CountyId`, as these are the columns in each df that store the county's FIPS Codes.

In [9]:
df_mvi_county.set_index('GEOID', inplace = True)
df_county_data.set_index('CountyId', inplace = True)

In [10]:
df_mvi_county.head(2)

,State,state_GEOID,county_name,mvi,Th1,Th2,Th3,Th4,Th5,Th6
GEOID,,,,,,,,,,
2282,Alaska,2,Yakutat,48.58,17.38,52.98,36.55,94.52,16.43,78.32
2060,Alaska,2,Bristol Bay,41.17,27.76,64.85,30.31,71.47,13.66,58.90


In [11]:
df_county_data.head(2)

,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
CountyId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,Alabama,Autauga County,55036,26899,28137,2.7,75.4,18.9,0.3,0.9,0.0,41016,55317,2838,27824,2024,13.7,20.1,35.3,18.0,23.2,8.1,15.4,86.0,9.6,0.1,0.6,1.3,2.5,25.8,24112,74.1,20.2,5.6,0.1,5.2
1003,Alabama,Baldwin County,203360,99527,103833,4.4,83.1,9.5,0.8,0.7,0.0,155376,52562,1348,29364,735,11.8,16.1,35.7,18.2,25.6,9.7,10.8,84.7,7.6,0.1,0.8,1.1,5.6,27.0,89527,80.7,12.9,6.3,0.1,5.5


### Now that the indexes are set as the columns with the FIPS Codes, ready to join the two dfs

In [12]:
df_joinsd = df_mvi_county.join(df_county_data, how = 'inner', lsuffix = '_mvi', rsuffix = '_cd')

In [13]:
df_joinsd.head(2)

,State_mvi,state_GEOID,county_name,mvi,Th1,Th2,Th3,Th4,Th5,Th6,State_cd,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
2282,Alaska,2,Yakutat,48.58,17.38,52.98,36.55,94.52,16.43,78.32,Alaska,Yakutat City and Borough,682,366,316,5.6,43.7,0.9,28.3,5.4,1.3,522,64583,11434,32393,4250,6.2,10.1,24.9,18.6,23.6,21.0,11.9,48.7,14.3,0.8,26.9,5.0,4.2,6.5,377,45.6,44.0,10.3,0.0,6.5
2060,Alaska,2,Bristol Bay,41.17,27.76,64.85,30.31,71.47,13.66,58.90,Alaska,Bristol Bay Borough,917,532,385,4.5,50.3,0.4,34.4,1.4,0.3,694,79500,10833,42002,4224,7.1,11.6,30.5,18.5,21.1,15.4,14.6,58.9,16.0,0.2,17.6,4.1,3.1,9.4,508,54.3,34.6,11.0,0.0,6.8


In [15]:
print(f'Verifying there are no null values in joined df. . . \nNull Count: {df_joinsd.isna().sum().sum()}')

Verifying there are no null values in joined df. . . 
Null Count: 0


## Step 2 | Cleaning the data